In [1]:
%matplotlib inline

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display
from datetime import date

# Data Loading

SF DBI’s development pipeline datasets use four nearly-identical data models. Parcel identificator was called “BLKLOT” or “Block Lot” in older datasets and is called APN (Assessor's Parcel Number) in the most recent datasets. The same parcel id can written with a leading zero in one dataset and without a leading zero in another one. Prior to 2014 Q3, datasets don’t have fields for affordable unit counts (total affordable units and net affordable units).

Some project records don’t include building permit id or use the placeholder “MULTIPLE” instead of actual permit references. Initially, we remove permitless projects from the dataset, than re-add them at a later stage.

In [2]:
def loadData(fileName, label, fmt = 1):
    columns = ['UNITS', 'NET_UNITS', 'AFF_UNITS', 'NET_AFF_UNITS', 'NAMEADDR', 'BP_APPLNO', 'BESTSTAT', 'BESTDATE', 'LOCATION']
    converters = { 'APN': lambda x: x[4:].lstrip('0'), 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 2 or fmt == 3:
        converters = { 'BLKLOT': lambda x: x.lstrip('0'), 'BP_APPLNO': lambda x: 'N'+x.replace(',','') }
    if fmt == 4:
        converters = { 'Block Lot': lambda x: x.lstrip('0'), 'DBI Permit': lambda x: 'N'+x.replace(',','')}
    if fmt == 6:
        converters = { 'APN': lambda x: x[4:].lstrip('0'), 'BPAPPLNO': lambda x: 'N'+x.replace(',','') }
        
    X = pd.read_csv(fileName, sep=',', parse_dates=[], infer_datetime_format=True, quotechar='"', converters=converters,dtype={
        'APN': str
    })
    
    if fmt == 2 or fmt == 3:
        X = X.rename(columns={"AFF_UNITS_NET": "NET_AFF_UNITS", "BLKLOT": "APN"})
    if fmt == 3:
        X = X.rename(columns={"UNITSNET": "NET_UNITS", "Geography": "LOCATION"})
    if fmt == 4:
        X['LOCATION'] = X['Location 1']#.apply(lambda x: x.split('\n')[1])
        X['NAMEADDR'] = X['Location 1'].apply(lambda x: x.split('\n')[0])
        
        X = X.rename(columns={"Units": "UNITS", "Net Added Units": "NET_UNITS", "Best Stat": "BESTSTAT", "Best Date": "BESTDATE", 'Block Lot': 'APN',
                             'DBI Permit': 'BP_APPLNO'})
        X['NET_AFF_UNITS'] = 0
        X['AFF_UNITS'] = 0
    if fmt == 5:
        X = X.rename(columns={"PROPUSE": "PROJECT_TYPE"})
    if fmt == 6:
        X = X.rename(columns={"UNITSNET": "NET_UNITS", "Location": "LOCATION", "BPAPPLNO": "BP_APPLNO", 
                              "AFFORDABLE": "AFF_UNITS", "AFFORDABLENET": "NET_AFF_UNITS"})
        X['NET_AFF_UNITS'] = X['NET_AFF_UNITS'].fillna(0)
        X['AFF_UNITS'] = X['AFF_UNITS'].fillna(0)
    X = X.rename(columns={"Location": "LOCATION"})
    X['LOCATION'] = X['LOCATION'].apply(lambda x: str(x).split('\n')[-1])

    X = X[X['UNITS'] > 0]
    X = X[X['NET_UNITS'] >= 0]
    
    # Filtering records without permits
    M = X[(X['BP_APPLNO'] == 'N') | (X['BP_APPLNO'] == 'NMULTIPLE')][columns+['BP_APPLNO']]
    X = X[(X['BP_APPLNO'] != 'N') & (X['BP_APPLNO'] != 'NMULTIPLE')]
    
    # Filtering duplicated records
    D = X[X.duplicated('APN', keep=False)][columns+['APN']].sort_values('APN')
    X=X[~X.duplicated('APN')]
    
    # Set unique index by Permit ID
    X = X.set_index('APN')
    X.index.names = ['APN']
    X['BESTDATE'] = pd.to_datetime(X["BESTDATE"])
    X['BESTSTAT'] = X["BESTSTAT"].str.upper()
    X[['UNITS', 'NET_UNITS', 'NET_AFF_UNITS', 'AFF_UNITS']] = X[['UNITS', 'NET_UNITS', 'NET_AFF_UNITS', 'AFF_UNITS']].astype(int)
    
    X = X[columns]
    
    return X,M,D

# Helper Indexes

y13q4 = 0
y14q1 = 1
y14q2 = 2
y14q3 = 3
y14q4 = 4
y15q1 = 5
y15q2 = 6
y15q3 = 7
y15q4 = 8
y16q1 = 9
y16q2 = 10
y16q3 = 11
y16q4 = 12
y17q1 = 13

# Files and versions

files = [
    {'label': '2013\'Q4', 'file': 'data/SF_Development_Pipeline_2013_Q4.csv', 'format': 4},
    {'label': '2014\'Q1', 'file': 'data/SF_Development_Pipeline_2014_Q1.csv', 'format': 4},
    {'label': '2014\'Q2', 'file': 'data/SF_Development_Pipeline_2014_Q2.csv', 'format': 4},
    {'label': '2014\'Q3', 'file': 'data/SF_Development_Pipeline_2014_Q3.csv', 'format': 4},
    {'label': '2014\'Q4', 'file': 'data/SF_Development_Pipeline_2014_Q4.csv', 'format': 3},
    {'label': '2015\'Q1', 'file': 'data/SF_Development_Pipeline_2015_Q1.csv', 'format': 2},
    {'label': '2015\'Q2', 'file': 'data/SF_Development_Pipeline_2015_Q2.csv', 'format': 1},
    {'label': '2015\'Q3', 'file': 'data/SF_Development_Pipeline_2015_Q3.csv', 'format': 1},
    {'label': '2015\'Q4', 'file': 'data/SF_Development_Pipeline_2015_Q4.csv', 'format': 1},
    {'label': '2016\'Q1', 'file': 'data/SF_Development_Pipeline_2016_Q1.csv', 'format': 1},
    {'label': '2016\'Q2', 'file': 'data/SF_Development_Pipeline_2016_Q2.csv', 'format': 5},
    {'label': '2016\'Q3', 'file': 'data/SF_Development_Pipeline_2016_Q3.csv', 'format': 1},
    {'label': '2016\'Q4', 'file': 'data/SF_Development_Pipeline_2016_Q4.csv', 'format': 1},
    {'label': '2017\'Q1', 'file': 'data/SF_Development_Pipeline_2017_Q1.csv', 'format': 1},
    {'label': '2017\'Q2', 'file': 'data/SF_Development_Pipeline_2017_Q2.csv', 'format': 6}
]

# Loading Data
count = len(files)
labels = []
data = []
missing = []
duplicates = []

for f in files:
    t,m,d = loadData(f['file'], f['label'], f['format'])
    data.append(t)
    missing.append(m)
    duplicates.append(d)
    labels.append(f['label'])

/anaconda/lib/python3.6/site-packages/ipykernel_launcher.py:12: ParserWarning: Both a converter and dtype were specified for column APN - only the converter will be used
  if sys.path[0] == '':


In [3]:
ids = ["3783001",
"3756003",
"3736074",
"4105009",
"4172022",
"8711031",
"0794028",
"3753106",
"3730015",
"0667016",
"4102026",
"4884025",
"3994001B",
"3542039",
"0629037",
"3719010",
"0671006",
"0785029",
"4041009",
"1073013",
"3703086",
"0857002",
"0619012",
"3518020",
"0807010",
"0814001",
"8715004",
"8711021",
"3703062",
"3738004",
"3703079",
"3751029",
"4167007",
"4059001A",
"3731001",
"6969001",
"2515001",
"3635014",
"1450008",
"3726103",
"0836011",
"0166003",
"3952001B",
"3702391",
"3706093",
"3708012",
"0331011",
"0670011",
"0620006",
"3575070",
"0311016",
"3549064",
"3753096",
"0101004",
"3576090",
"3617008",
"3197010",
"3985024",
"0317005",
"3555036",
"1153020",
"1609021F",
"4022022",
"3511066",
"0516028A",
"1412019",
"1456023",
"6531022",
"2374019",
"5826002",
"5926021",
"5319037",
"2628026",
"1152018",
"6517022",
"0818048",
"3753076",
"4040025",
"7106058",
"5683012",
"7106059",
"0519029",
"3656010",
"3612017",
"1505014",
"1068003",
"4095018",
"1736033",
"0840001",
"3612017A",
"3116007",
"3636027",
"1286024",
"5577012",
"0539031",
"0948018",
"6942028",
"6087025",
"6020033A",
"5896022",
"6758032",
"2709065",
"0069011",
"1419007",
"0937022",
"6604039",
"1628028",
"2701A019",
"1542005",
"2823A016",
"2317016G",
"2184034",
"6086012C",
"4004034",
"4274001",
"5306025",
"2993088",
"0181013",
"6771032",
"1569002",
"6959030",
"3618071",
"1835010",
"1055021",
"1524018A",
"5721077",
"4937018",
"0069020",
"2688072",
"6659006",
"1253009",
"6521025",
"0520008",
"5556019",
"2349038",
"1254019",
"3545097",
"3755049",
"5338025",
"4262020A",
"1383004",
"4209002",
"3548021",
"3631017",
"1667024",
"0867031",
"6374028",
"4013007",
"5943008",
"5529012",
"0482030",
"2038A013",
"3139013",
"7008038",
"3726062",
"0073018B",
"1890005",
"1210031",
"2393002",
"0945012",
"6577046",
"3116006",
"2623028",
"6968008",
"1564013",
"5892032",
"0693015",
"5643035",
"5971029",
"4275003",
"7173011",
"2038A018",
"2861011",
"4210013",
"4282A023",
"1405045",
"7107023",
"0226007",
"2416002H",
"5943050",
"5410008",
"0829023",
"3576039",
"1729012",
"5476014",
"1511034",
"5338018",
"6581060",
"2829010",
"1269167",
"4084018",
"7044A018",
"5858024",
"5403001",
"3148014A",
"1781033",
"6554037",
"5744016",
"1561038A",
"1675016",
"1371039",
"1869011",
"0041037",
"2370054",
"5324020",
"6717008",
"6013009",
"6464037",
"1505027",
"0459007",
"4591C143",
"0028014",
"3995022",
"4591C165",
"0503021",
"1084001B",
"3510028",
"1439021",
"0158010",
"3615047",
"3642041",
"5640010",
"3537025",
"6506032",
"3654040",
"6534029",
"0105065",
"3532048",
"3727049",
"1168040",
"3613053",
"4008003",
"1462031",
"4148027",
"2630018",
"1265019",
"0498020",
"1246018",
"1456003",
"3588001C",
"0156015A",
"4211011",
"0856013",
"0156036",
"4218036",
"1732037",
"5891004A",
"5613064",
"5329009",
"5406016",
"0519012",
"6435012",
"5909023",
"0515031",
"5322047",
"5259034",
"4275001A",
"1730037",
"5386002",
"7552026",
"6786025",
"1067009",
"1066030",
"3730118",
"5618007",
"6604038",
"4071006",
"3648033",
"0944030",
"1416003",
"5654023",
"1355004",
"1050007",
"0480013",
"6511018",
"2630017",
"3561023",
"0441B007",
"3563015B",
"1533029",
"7295018",
"3579022",
"1550045",
"1043009",
"1549034",
"1510013",
"5613063",
"1407046",
"3541083",
"3587058",
"6738020",
"0119017",
"2105030",
"2416013",
"7537002",
"6618003",
"2187001E",
"3621081",
"1444015",
"5557021",
"1423025",
"1104001",
"6513053",
"3120009",
"2614048",
"264603"]

In [6]:
# ids = ['3783001']
results = []
for i in range(len(ids)):
    apn = ids[i].lstrip('0')
    filed = None
    approved = None
    for j in range(count):
        if apn in data[j].index.values:
            state = data[j].loc[apn]['BESTSTAT']
            stateDate = data[j].loc[apn]['BESTDATE']
            if state == 'PL FILED':
                if filed is None:
                    filed = stateDate
            if state == 'PL APPROVED':
                if approved is None:
                    approved = stateDate
    results.append({'apn': apn, 'filed': filed, 'approved': approved})
    # data[ids[i]] = { 'filed': None, 'approved': None, 'status': None }
print(results)

[{'apn': '3783001', 'filed': None, 'approved': None}, {'apn': '3756003', 'filed': None, 'approved': None}, {'apn': '3736074', 'filed': None, 'approved': None}, {'apn': '4105009', 'filed': None, 'approved': None}, {'apn': '4172022', 'filed': None, 'approved': None}, {'apn': '8711031', 'filed': None, 'approved': None}, {'apn': '794028', 'filed': None, 'approved': None}, {'apn': '3753106', 'filed': None, 'approved': None}, {'apn': '3730015', 'filed': None, 'approved': None}, {'apn': '667016', 'filed': None, 'approved': None}, {'apn': '4102026', 'filed': None, 'approved': None}, {'apn': '4884025', 'filed': None, 'approved': None}, {'apn': '3994001B', 'filed': None, 'approved': None}, {'apn': '3542039', 'filed': None, 'approved': None}, {'apn': '629037', 'filed': None, 'approved': None}, {'apn': '3719010', 'filed': None, 'approved': None}, {'apn': '671006', 'filed': None, 'approved': None}, {'apn': '785029', 'filed': None, 'approved': None}, {'apn': '4041009', 'filed': None, 'approved': Non

In [7]:
filedCount = 0
approvedCount = 0
for f in range(len(results)):
    print(results[f])
    if results[f]['filed'] is not None:
        filedCount += 1
    if results[f]['approved'] is not None:
        approvedCount += 1
print("Filed {}/{} of {}".format(filedCount, approvedCount, len(results)))

{'apn': '3783001', 'filed': None, 'approved': None}
{'apn': '3756003', 'filed': None, 'approved': None}
{'apn': '3736074', 'filed': None, 'approved': None}
{'apn': '4105009', 'filed': None, 'approved': None}
{'apn': '4172022', 'filed': None, 'approved': None}
{'apn': '8711031', 'filed': None, 'approved': None}
{'apn': '794028', 'filed': None, 'approved': None}
{'apn': '3753106', 'filed': None, 'approved': None}
{'apn': '3730015', 'filed': None, 'approved': None}
{'apn': '667016', 'filed': None, 'approved': None}
{'apn': '4102026', 'filed': None, 'approved': None}
{'apn': '4884025', 'filed': None, 'approved': None}
{'apn': '3994001B', 'filed': None, 'approved': None}
{'apn': '3542039', 'filed': None, 'approved': None}
{'apn': '629037', 'filed': None, 'approved': None}
{'apn': '3719010', 'filed': None, 'approved': None}
{'apn': '671006', 'filed': None, 'approved': None}
{'apn': '785029', 'filed': None, 'approved': None}
{'apn': '4041009', 'filed': None, 'approved': None}
{'apn': '1073013